In [111]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [112]:
# Load the dataset
df = pd.read_csv('..//dataset//final_dataset_compressed.csv')  # Replace with your dataset path

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']



# scaler=RobustScaler()
# X_scaled = scaler.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [113]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(320000, 1, 6) (320000,) (400000, 1, 6)


# Model

In [114]:
def build_deep_rnn_bi_lstm():
    model = Sequential()
    
    # Add multiple SimpleRNN layers for Deep RNN
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(SimpleRNN(units=50, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(SimpleRNN(units=50, return_sequences=True))
    model.add(Dropout(0.1))

    # Add multiple Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(units=32)))
    model.add(Dropout(0.1))

    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])

    return model


In [115]:
# Early stopping callback
early_stopping = EarlyStopping(
    monitor='mse',  
    patience=3,   
    restore_best_weights=True, 
    verbose=1
)
# Model checkpoint callback
best_weights_filepath ='..//models//best_model.keras'

model_checkpoint = ModelCheckpoint(
    best_weights_filepath,  
    monitor='val_mse',  
    mode='min',       
    save_best_only=True,   
    verbose=1
)

# Instantiate and train the model
model = build_deep_rnn_bi_lstm()
history = model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.2, verbose=1,   
 callbacks=[early_stopping]
)

Epoch 1/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0934 - mae: 0.0654 - mse: 0.0303 - val_loss: 0.0316 - val_mae: 0.0202 - val_mse: 0.0107
Epoch 2/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.0328 - mae: 0.0216 - mse: 0.0109 - val_loss: 0.0301 - val_mae: 0.0207 - val_mse: 0.0102
Epoch 3/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.0314 - mae: 0.0210 - mse: 0.0105 - val_loss: 0.0325 - val_mae: 0.0209 - val_mse: 0.0110
Epoch 4/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.0316 - mae: 0.0212 - mse: 0.0106 - val_loss: 0.0293 - val_mae: 0.0194 - val_mse: 0.0100
Epoch 5/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.0314 - mae: 0.0212 - mse: 0.0107 - val_loss: 0.0288 - val_mae: 0.0201 - val_mse: 0.0098
Epoch 6/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.0303 - mae: 0.0205 - mse: 0.0102 - val_loss: 0.0286 - val_mae: 0.0193 - val_mse: 0.0097
Epoch 7/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.0301 - mae: 0.0204 - ms

In [116]:
# import numpy as np

# # Set numpy to display full float numbers
# np.set_printoptions(suppress=True, precision=10)

In [117]:
# results = model.evaluate(X_test, y_test, verbose=1)
# loss, mse, mae = results  # Unpack all the returned metrics
# print(f'Loss: {loss:.4f}')
# print(f'Mean Squared Error (MSE): {mse:.4f}')
# print(f'Mean Absolute Error (MAE): {mae:.4f}')


In [118]:

from sklearn.metrics import mean_squared_error
import numpy as np

# Make predictions
y_pred = model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')

2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
Mean Squared Error (MSE): 0.0094541935250163
Root Mean Squared Error (RMSE): 0.0972326772490417


In [119]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()


In [120]:
#saving the model
# model.save('..//models//deep_rnn_bi_lstm_updated_1.keras')  # Replace with your model path